# Layers

> This module defines the core pytorch layers used for learning from phase response data

In [17]:
#| default_exp layers

In [18]:
#| hide
from nbdev.showdoc import *

In [19]:
#| export

import torch
import torch.nn as nn
from torch.nn.modules import linear
import torch.optim as optim
import pytorch_lightning as pl
import pylab as plt
from torch.utils.data import DataLoader, Dataset
import torchcde
import numpy as np
from torchdiffeq import odeint as odeint
from typing import List, Tuple, Union


In [20]:


class PolarModel(nn.Module):

    def __init__(self,
                 tau: float = 24.0,
                 K: float = 0.30,
                 beta: float = 0.02,
                 gamma: float = 0.1,
                 A = [1.0, 0.4],
                 B = [0.0, 0.3],):
        super().__init__()
        self.omega = torch.tensor(2.0*torch.pi/tau)
        self.K = torch.tensor(K)
        self.gamma = torch.tensor(gamma)
        self.beta = torch.tensor(beta)
        self.pulse = lambda t: 0.0
        self.A = torch.tensor(A)
        self.B = torch.tensor(B)
        self.sigma = 0.0

    def set_pulse(self, f):
        self.pulse = f

    def forward(self, t, state):
        R = state[0]
        psi = state[1]

        pulseAmp = self.A[0]*0.5*(1.0-R**4.0)*torch.cos(
            psi+self.B[0])+self.A[1]*0.5*R*(1.0-R**8.0)*torch.cos(2.0*psi+self.B[1])
        pulsePhase = self.sigma-self.A[0]*0.5*(R**3+1.0/R)*torch.sin(
            psi+self.B[0])-self.A[1]*0.5*(1.0+R**8)*torch.sin(2.0*psi+self.B[1])

        dR = - self.gamma*R+self.K * \
            torch.cos(self.beta)/2.0*R*(1-R**2) + self.pulse(t)*pulseAmp
        dPsi = self.omega + self.K * \
            torch.sin(self.beta)/2.0*(1+R**2) + self.pulse(t)*pulsePhase
        dfunc = torch.hstack((dR, dPsi)).view(state.shape)
        return dfunc

    def to_phase(self, trajectory):
        return trajectory[:, 1]



In [21]:
#| export


class MicoscopicModel(nn.Module):

    def __init__(self, N: int = 100):
        super().__init__()

        # Fittable Parameters
        self.tau = torch.nn.Parameter(0.02*torch.randn(N)+24.0)
        self.K = torch.nn.Parameter(torch.rand(N, N))
        self.A = torch.nn.Parameter(torch.zeros(2))
        self.B = torch.nn.Parameter(torch.zeros(2))
        self.beta = torch.nn.Parameter(torch.tensor(0.0))
        self.N = N
        self.forcing = None

    def forward(self, t, state):
        phi = state[..., :]  # batch vector of length N
        lval = self.forcing.evaluate(t)[1]
        Q = self.A[0]*torch.sin(phi)+self.A[1]*torch.sin(
            2.0*phi)+self.B[0]*torch.cos(phi)+self.B[1]*torch.cos(2*phi)

        correctionTerm = torch.diag(self.K) * torch.sin(self.beta)
        dphi = 2*np.pi/self.tau+1.0/self.N * \
            torch.sum(
                self.K*torch.sin(phi[None, :]-phi[:, None]+self.beta), axis=0)+Q*lval - correctionTerm
        dfunc = dphi.view(state.shape)
        return dfunc

    def update_coeff(self, coeffs, time_coeffs):
        self.forcing = torchcde.CubicSpline(coeffs, t=time_coeffs)

    def __repr__(self) -> str:
        my_rep = ""
        my_rep += "\n KNN with parameters: \n"
        for name, param in self.named_parameters():
            if param.requires_grad:
                my_rep += "Parameter " + \
                    str(name) + ", has value: " + str(param.data) + "\n"
        return my_rep

    def to_dict(self):
        """
            Create a dictionary of the fit parameters for a spm object
        """
        return {name: param.data.item() for name, param in self.named_parameters()}



In [22]:
#| export

class SmallPerturbationModel(nn.Module):

    def __init__(self, 
                 tau: float = 24.0, 
                 inner_layers_forcing: int = 10,
                 inner_layers_autonmous: int = 10,
                 use_nn_perturbation: bool = True,
                 use_nn_autonomous: bool = False):
        super().__init__()

        # Fit able Parameters, init from zero values to avoid nan
        self.tau = torch.nn.Parameter(torch.tensor(tau))
        self.K = torch.nn.Parameter(torch.tensor(0.3))
        self.A = torch.nn.Parameter(torch.zeros(2))
        self.B = torch.nn.Parameter(torch.zeros(2))
        self.sigma = torch.nn.Parameter(torch.tensor(0.0))
        self.gamma = torch.nn.Parameter(torch.tensor(0.05))
        self.beta = torch.nn.Parameter(torch.tensor(0.0))

        self.use_nn_perturbation = use_nn_perturbation
        self.use_nn_autonomous = use_nn_autonomous
        
        if self.use_nn_perturbation:
            self.neuralNetwork = nn.Sequential(nn.Linear(2, inner_layers_forcing),
                                            nn.LeakyReLU(),
                                            nn.Linear(inner_layers_forcing,
                                                        inner_layers_forcing),
                                            nn.LeakyReLU(),
                                            nn.Linear(inner_layers_forcing, 2))
            #self.neuralNetwork.apply(self.init_weights_zero)
        
        if self.use_nn_autonomous:
            self.autonomousTerm = nn.Sequential(nn.Linear(2, inner_layers_autonmous),
                                            nn.LeakyReLU(),
                                            nn.Linear(inner_layers_autonmous,
                                                        inner_layers_autonmous),
                                            nn.LeakyReLU(),
                                            nn.Linear(inner_layers_autonmous, 2))
            self.autonomousTerm.apply(self.init_weights_zero)
        
    def forward(self, 
                t : float, 
                state : torch.Tensor, 
                autonomous_nn: bool = True, 
                perturbation_nn: bool=True):
        PulseValue = self.forcing.evaluate(t)[1]
        R = state[..., 0]
        Psi = state[..., 1]

        LightAmp = self.A[0]*0.5*(1.0-R**4.0)*torch.cos(Psi+self.B[0]) + \
            self.A[1]*0.5*R*(1.0-R**8.0)*torch.cos(2.0*Psi+self.B[1])
        LightPhase = self.sigma-self.A[0]*0.5*(R**3+1.0/R)*torch.sin(
            Psi+self.B[0])-self.A[1]*0.5*(1.0+R**8)*torch.sin(2.0*Psi+self.B[1])

        dR = -1.0*self.gamma*R+self.K * \
            torch.cos(self.beta)/2.0*R*(1.0-R**4)+LightAmp*PulseValue
        dPsi = 2*torch.pi / self.tau+self.K/2.0 * \
            torch.sin(self.beta)*(1+R**4)+LightPhase*PulseValue

        dfunc = torch.hstack((dR, dPsi)).view(state.shape)
        # Add in the neural network terms, if they are used
        if self.use_nn_perturbation and perturbation_nn:
            neural_term_perturbation = self.neuralNetwork(torch.sin(state.float()))
            dfunc += neural_term_perturbation.tanh()*PulseValue
        if self.use_nn_autonomous and autonomous_nn:
            autonomous_term_perturbation = self.autonomousTerm(torch.sin(state.float()))
            dfunc += autonomous_term_perturbation.tanh()
        return dfunc

    def update_coeff(self, coeffs, time_coeffs):
        self.forcing = torchcde.CubicSpline(coeffs, t=time_coeffs)
        
    def init_weights_zero(self, m):
        if isinstance(m, nn.Linear):
            torch.nn.init.zeros_(m.weight)
            m.bias.data.fill_(0.0)

    def __repr__(self) -> str:
        my_rep = ""
        my_rep += "\n Small Perturbation Model with parameters: \n"
        for name, param in self.named_parameters():
            if param.requires_grad:
                my_rep += "Parameter " + \
                    str(name) + ", has value: " + str(param.data) + "\n"
        return my_rep

    def to_dict(self) -> dict:
        return {name: param.data.item() for name, param in self.named_parameters()}






In [23]:

#| export 


class PureANN(nn.Module):

    def __init__(self, tau: float = 24.0):
        super().__init__()

        self.tau = tau
        self.autonomousANN = torch.nn.Sequential(
            torch.nn.Linear(2, 10),
            torch.nn.LeakyReLU(),
            nn.Linear(10, 10),
            torch.nn.LeakyReLU(),
            nn.Linear(10, 2))
        self.pulseANN = torch.nn.Sequential(
            torch.nn.Linear(2, 10),
            torch.nn.LeakyReLU(),
            nn.Linear(10, 10),
            torch.nn.LeakyReLU(),
            nn.Linear(10, 2))

    def forward(self, t, state):
        Bhat = self.forcing.evaluate(t)[1]
        pulseTerm = Bhat*self.pulseANN(state)
        autoTerm = torch.tensor([0.0, 2*torch.pi/self.tau]) + \
            self.autonomousANN(torch.sin(state))
        return (autoTerm + pulseTerm).tanh()

    def update_coeff(self, coeffs, time_coeffs):
        self.forcing = torchcde.CubicSpline(coeffs, t=time_coeffs)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=1.0)
            if module.bias is not None:
                module.bias.data.zero_()

    def __repr__(self) -> str:
        my_rep = ""
        my_rep += "\n Pure ANN with parameters: \n"
        for name, param in self.named_parameters():
            if param.requires_grad:
                my_rep += "Parameter " + \
                    str(name) + ", has value: " + str(param.data) + "\n"
        return my_rep

    def to_dict(self):
        """
            Create a dictionary of the fit parameters for a spm object
        """
        return {name: param.data.item() for name, param in self.named_parameters()}


In [24]:
#| hide
import nbdev; nbdev.nbdev_export()